In [17]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore")
import re
import time
import requests

In [46]:
url = '''https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.recommend_order&years=-1&locations=all'''
driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [40]:
# 개발섹터 선택
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/button/span[2]').click()

In [42]:
# 직무선택 (5개씩)
# num = 2 ~ 38
driver.find_element(By.XPATH, f'//*[@id="__next"]/div[3]/article/div/div[2]/section/div[2]/div/button[{num}]').click()

# 선택완료
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[3]/button/span[2]').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="__next"]/div[3]/article/div/div[2]/section/div[2]/div/button[2]"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7CF417012+3522402]
	(No symbol) [0x00007FF7CF038352]
	(No symbol) [0x00007FF7CEEE5ABB]
	(No symbol) [0x00007FF7CEF2BF0E]
	(No symbol) [0x00007FF7CEF2C08C]
	(No symbol) [0x00007FF7CEF6E437]
	(No symbol) [0x00007FF7CEF4F09F]
	(No symbol) [0x00007FF7CEF6BDA3]
	(No symbol) [0x00007FF7CEF4EE03]
	(No symbol) [0x00007FF7CEF1F4D4]
	(No symbol) [0x00007FF7CEF205F1]
	GetHandleVerifier [0x00007FF7CF449B9D+3730157]
	GetHandleVerifier [0x00007FF7CF49F02D+4079485]
	GetHandleVerifier [0x00007FF7CF4975D3+4048163]
	GetHandleVerifier [0x00007FF7CF16A649+718233]
	(No symbol) [0x00007FF7CF044A3F]
	(No sym

In [25]:
driver.find_element(By.XPATH, '//*[@id="__next"]/div[3]/article/div/div[2]/section/div[2]/div/button[2]').click()